# Intro

The task in hand is to correctly identify the digits in the given pictures of handwritten digits. Handwritten digit recognition is one of the oldest problem in computer vision and often used to benchmark classification algorithms. In this notebook, I will create a simple ANN architecture using PyTorch and train and test its accuracy.

In [3]:
import numpy as np
import matplotlib.pyplot as plt

In [4]:
import torch
from torch import nn, optim
from torch.nn import Sequential
from torchvision import datasets, transforms

In [2]:
print(torch.__version__)

1.4.0+cpu
